## EIP Project


### Problem Statement: Use only Winograd Conv and convert this into Tutorial (train as well)
### https://github.com/zlpure/Facial-Expression-Recognition 

Before we get to the final code lets dive into the Winograd Convolutions.

### Winograd Convolutions:

Inspite of reading many papers on Winograd algorithm I still don't get how its works. These are the sourcs I refered to 
- https://arxiv.org/pdf/1509.09308.pdf
- https://www.scribd.com/doc/55802885/Winograd-algorithm

What I've gathered from all these sources has been summerized here:

#### What are Winograd Convolutions:
Winograd Convolutions uses the Winograd minimal filtering algorithm. The key idea is to perform convolution in transformed domains using Winograd algorithm. This algorithm reduces the number of multiplications with the expense of additional addition and constant multiplication.
According to the article https://ai.intel.com/winograd-2/, The Winograd algorithm works on small tiles of the input image. In a nutshell, the input tile and filter are transformed, the outputs of the transform are multiplied together in an element-wise fashion, and the result is transformed back to obtain the outputs of the convolution.

#### Why do we need Winograd Conv? 
In most of the applicaions of deep neural networks, the speed takes the priority over precision. E.g. in self-driving cars.

Other references:
- http://cs231n.stanford.edu/reports/2016/pdfs/117_Report.pdf
- https://arxiv.org/pdf/1803.09004.pdf
- https://www.encyclopediaofmath.org/index.php/Winograd_small_convolution_algorithm

#### Where to find the Winograd Conv?
Since I was not confident of my understanding in Winograd algorithms I couldn't implement it. There were few implementations of the algorithm. I found these options which were directly usable:
- One is Nervana Neon: Implementation of Winograd Conv in this architecture seemed straight forward. But I did not find a way of comparing the speed with and without Winograd Conv with this implementation.
https://github.com/NervanaSystems/neon/tree/master/neon

- Then there is CudNN which implements Winograd Conv. The Winograd Conv is enabled by default in version higher than 5. But it  also provides an environment variable TF_ENABLE_WINOGRAD_NONFUSED, that could be used to enable or disable it. So I chose this to find out how Winograd Conv can help the performance of Deep neural networks.
https://docs.nvidia.com/deeplearning/dgx/tensorflow-user-guide/index.html#tf_enable_winograd_nonfused


### Using Winograd Convolutions in CuDNN 
I used Google Colaboratory, which did not have Cuda and CuDNN installed. Although I was expecting Winograd Conv to speed up the training, the prep took a lot of time. Everytime I log in I had to install Cuda, and CuDNN that took a lot of time. 
I tested the speed for just one epoch, supposing that it should be enough to measure the difference when Wino-Conv is on and off.
I did not find much difference in either case. 

The set-up for Cuda 8 CuDNN 6
https://github.com/Curiousss/InkerIntern/blob/master/FER_WINO_SEPARABLE.ipynb

This notebook installs CUDA 9 and CuDNN 7 in colaboratory.
It also re-installs tensorflow-gpu to make sure that the CuDNN is used. 
The model is test with the enironment variable TF_ENABLE_WINOGRAD_NONFUSED set to "1" and "0". The speed was almost the same. Any small differences in the speed were not found related to the setting of the flag. Sometimes enabling the flag was faster sometime disabling was faster.
https://github.com/Curiousss/InkerIntern/blob/master/CUda9Cudnn7.ipynb

I think I spent almost 80-90% of the time on Wino-Conv so it was very difficult to let go since I felt I did not go anywhere with it. First of all I did not get the math, and then implementation did not seem to improve the speed.


### Model for Facial Emotion Recognition
First I ran implementation given in https://github.com/zlpure/Facial-Expression-Recognition using the model.json and weights given. The accuracy was 64% and with Image augmentation it went upto 66-67%. The speed was around 77s per epoch.
Find the implementation here: https://github.com/Curiousss/InkerIntern/blob/master/FacialEmotion.ipynb

I re-implemented the same model at first. Then enhanced with these changes:

- Separable Convolutions: To speed up the model the regular convolutions was replaced with SeparableConv2D. The speed almost doubled. They have fewer parameters than regular convolutional layers, and thus are less prone to overfitting. With fewer parameters, they also require less operations to compute, and thus are cheaper and faster.
- The model with 7x7 and 5x5 layers did not help in any kind of improvement. Only 3x3 convolutions were retained. Two 3x3 conv layers have a receptive field of 5x5, and have fewer mathematical operations and more non-linearities. So they should be faster and able to create more complex functions.
- Global Average Pooling: The fully-connected layers were replaced with Global Average Pooling. This increased the accuracy and the speed. GAP helps in minimizing overfitting reducing the total number of parameters in the model. 
- Image Augmentation: Applying random transformation on the image might actually hinder the training process. Hence each transformation was tested for its efficiecy and then chosen for the final model. 
    - Applying horizontal flip as very intuitive and it did improve the performance.
    - Applying vertical flip was not very helpful.
- The Dropout layers were removed since ImageAugmentation and GAP are already regularizing the network.
- Batch-size
Find the implementation of new model here:
https://github.com/Curiousss/InkerIntern/blob/master/FER_WINO_SEPARABLE_NO_CUDNN.ipynb